# Here we try a variety of probes

They all fit into a sklearn style api, except they take torch tensors of >2 dims


In [1]:
import os
os.environ['TQDM_DISABLE'] = '1'


In [2]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

plt.style.use("ggplot")

from typing import Optional, List, Dict, Union
from jaxtyping import Float
from torch import Tensor

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path
from einops import rearrange


import datasets
from datasets import Dataset

from loguru import logger


from typing import Optional, List, Dict, Union, Tuple
from jaxtyping import Float
from torch import Tensor

logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")

# # quiet please
torch.set_float32_matmul_precision("medium")
import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")


In [3]:
# load my code
%load_ext autoreload
%autoreload 2

import lightning.pytorch as pl
from src.datasets.dm import DeceptionDataModule
from src.models.pl_lora_ft import AtapterFinetuner

from src.config import ExtractConfig
from src.prompts.prompt_loading import load_preproc_dataset, load_preproc_datasets
from src.models.load import load_model
from src.helpers.torch_helpers import clear_mem
from src.models.phi.model_phi import PhiForCausalLMWHS
# from src.eval.interventions import check_lr_intervention_predictive
from src.probes.utils import postproc


In [4]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# plt.style.use("ggplot")

plt.style.use(['seaborn-v0_8', 'seaborn-v0_8-paper'])


In [5]:
from src.eval.labels import ranking_truth_telling, ds2label_model_truth
from src.eval.ds import filter_ds_to_known

from src.probes.pl_ranking_probe import PLConvProbeLinear
from src.helpers.lightning import read_metrics_csv
from sklearn.metrics import roc_auc_score, accuracy_score
from src.helpers.pandas_classification_report import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, TensorDataset

from functools import partial
from src.eval.labels import ranking_truth_telling, undo_ranked_truth_telling
from src.helpers.ds import train_test_split_ds


?# Params

In [6]:
max_epochs = 50
batch_size=16
verbose = True
MAX_SAMPLES = 2600
SKIP=1
STRIDE=2
DECIMATE=1


## Setup data

We load data, previously collected, from the model


In [7]:
# !ls -altrh '/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/'
# # !ls -altrh '/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/ds_valtest_8c031b4aa03ae4d2'
# /media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/notebooks/lightning_logs/version_4/

BASE_FOLDER = Path("/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/notebooks/lightning_logs/version_4/")


In [10]:
f1_val = next(iter(BASE_FOLDER.glob('hidden_states/.ds/ds_valtest_*')))
f1_ood = next(iter(BASE_FOLDER.glob('hidden_states/.ds/ds_OOD_*')))
f1_val, f1_ood


(PosixPath('/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/notebooks/lightning_logs/version_4/hidden_states/.ds/ds_valtest_228c8c7509d7d7d2'),
 PosixPath('/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/notebooks/lightning_logs/version_4/hidden_states/.ds/ds_OOD_e52cbd9274aede8b'))

In [11]:
# load hidden state from a previously loaded adapter
# the columns with _base are from the base model, and adapt from adapter
# FROM TRAINING TRUTH
ds_val = Dataset.from_file(str(f1_val)).with_format("torch")
ds_oos = Dataset.from_file(str(f1_ood)).with_format("torch")

ds_out = datasets.interleave_datasets([ds_val, ds_oos], seed=42, 
                                    #   probabilities=[0.5, 0.5]
                                      )
ds_known1 = filter_ds_to_known(ds_out, verbose=True)
ds_known1 = ds_out



/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.venv/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


select rows are 74.58% based on knowledge


In [12]:
def flip_even_rows(row: dict, idx: int) -> dict:
    """this will flip adapter and base keys for even rows, this balances the task."""
    # also record the original rows, and which ones are flipped?
    row2 = {k.replace('_base', '_base2').replace('_adapt', '_adapt2'): v for k, v in row.items()}
    row = {**row2, **row}
    
    if idx%2 == 0:
        row = {k.replace('_base', '_tmp'): v for k, v in row.items()}
        row = {k.replace('_adapt', '_base'): v for k, v in row.items()} 
        row = {k.replace('_tmp', '_adapt'): v for k, v in row.items()}
    return row

ds_known = ds_known1.map(flip_even_rows, with_indices=True)
ds_known


Map:   0%|          | 0/2004 [00:00<?, ? examples/s]

Dataset({
    features: ['end_logits_base', 'choice_probs_base', 'binary_ans_base', 'label_true_base', 'label_instructed_base', 'instructed_to_lie_base', 'sys_instr_name_base', 'example_i_base', 'ds_string_base', 'template_name_base', 'correct_truth_telling_base', 'correct_instruction_following_base', 'end_residual_Wqkv_base', 'end_residual_fc1_base', 'end_logits_adapt', 'choice_probs_adapt', 'binary_ans_adapt', 'label_true_adapt', 'label_instructed_adapt', 'instructed_to_lie_adapt', 'sys_instr_name_adapt', 'example_i_adapt', 'ds_string_adapt', 'template_name_adapt', 'correct_truth_telling_adapt', 'correct_instruction_following_adapt', 'end_residual_Wqkv_adapt', 'end_residual_fc1_adapt', 'end_logits_adapt2', 'choice_probs_adapt2', 'binary_ans_adapt2', 'label_true_adapt2', 'label_instructed_adapt2', 'instructed_to_lie_adapt2', 'sys_instr_name_adapt2', 'example_i_adapt2', 'ds_string_adapt2', 'template_name_adapt2', 'correct_truth_telling_adapt2', 'correct_instruction_following_adapt2', '

## Importance matrix?

In [13]:
from src.probes.importance_matrix import get_importance_matrix

f = f"{BASE_FOLDER}/checkpoint_last/adapter_model.safetensors"
# +1 because the residuals are a product of diff, so the first row didn't have a diff. So we skip that corresponding first row of the importance matrix
importance_matrix = get_importance_matrix(f, layers=['fc1', 'Wqkv'])[SKIP+1::STRIDE, ::DECIMATE]
print(importance_matrix.shape)
plt.hist(importance_matrix.flatten(), bins=55);


FileNotFoundError: No such file or directory: "/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/notebooks/lightning_logs/version_4/checkpoint_last/adapter_model.safetensors"

### split datasets

we want to split by insample (from training) and out of distribution

In [ ]:
pd.Series(ds_val['ds_string_base']).value_counts(), pd.Series(ds_oos['ds_string_base']).value_counts()


In [ ]:
# seperate in sample and out of sample datasets
insample_datasets = list(set(ds_val['ds_string_base']))
outsample_datasets = list(set(ds_oos['ds_string_base']))
print(insample_datasets, outsample_datasets)

trainval_datasets = insample_datasets + outsample_datasets[:1]
test_datasets = outsample_datasets[1:]
trainval_datasets, test_datasets


In [ ]:
ds_trainval = ds_known.filter(lambda s: s in trainval_datasets, input_columns=['ds_string_base'])
ds_test = ds_known.filter(lambda s: s not in trainval_datasets, input_columns=['ds_string_base'])


MAX_SAMPLES = min(len(ds_trainval), MAX_SAMPLES)
ds_trainval = ds_trainval.select(range(MAX_SAMPLES))

MAX_SAMPLES = min(len(ds_test), MAX_SAMPLES)
ds_test = ds_test.select(range(MAX_SAMPLES))


len(ds_trainval), len(ds_test)


In [ ]:

# ds_train, ds_val = train_test_split_ds(ds_trainval, test_size=0.3, stratify_columns=['ds_string_base'])
# len(ds_train), len(ds_val)


In [ ]:
# set labels
ds2proxy = ranking_truth_telling
proxy2label = undo_ranked_truth_telling

# unit test labels
y_val1 = ds2proxy(ds_val)
y1 = proxy2label(y_val1, ds_val)
np.testing.assert_array_equal(y1, ds_val['label_true_base'], err_msg="undo_distance_truth_telling failed")
# assert (y1 == ds_val['label_true_base']).all()


In [ ]:
def ds2xy(ds: Dataset, ds2proxy) -> Tuple[Tensor, Tensor]:
    # x = torch.stack([ds['end_residual_stream_base'], ds['end_residual_stream_adapt']], -1)[:, SKIP::STIDE]
    X1 = torch.stack([ds['end_residual_fc1_base'], ds['end_residual_fc1_adapt']], dim=-1)
    X2 = torch.stack([ds['end_residual_Wqkv_base'], ds['end_residual_Wqkv_adapt']], dim=-1)
    x = torch.concat([X1, X2], dim=2)[:, SKIP::STRIDE, ::DECIMATE] # [layers, features, base/adapt]
    x = x * importance_matrix[None, :, :, None]**2
    y = ds2proxy(ds)
    return x, y


In [ ]:
xx = torch.tensor([[1.1, 1.3, 2.4]])
torch.diff(xx, dim=1)
# xx


In [ ]:
ds = ds_trainval
# X1 = torch.stack([ds['end_residual_fc1_base'], ds['end_residual_fc1_adapt']], dim=-1)
# X2 = torch.stack([ds['end_residual_Wqkv_base'], ds['end_residual_Wqkv_adapt']], dim=-1)
# x = torch.concat([X1, X2], dim=2)[:, SKIP::STIDE] # [layers, features, base/adapt]
# y = ds2proxy(ds)
# x.shape

X, y = ds2xy(ds, ds2proxy)
X.shape, y.shape


In [ ]:
from src.probes.lr import TorchRobustScaler, TorchLogisticRegression, TorchDummyClassifier
from src.eval.pl_sk import eval_ranking, analyze_dfres


## Probes


### LogisticRegression

In [ ]:
model = TorchLogisticRegression(random_state=42, 
                                max_iter=1000,
                                 class_weight='balanced',)
df_res = eval_ranking(model, ds_trainval, ds_test, proxy2label, ds2proxy)
df_res2 = analyze_dfres(df_res)


### Baseline: DummyClassifier


In [ ]:
model = TorchDummyClassifier(random_state=42, strategy="most_frequent")
df_res = eval_ranking(model, ds_trainval, ds_test, proxy2label, ds2proxy)
analyze_dfres(df_res);


### PL Conv MSE Ranking


In [ ]:

def distance_truth_telling(ds):
    """label whether the adapter or the base model were more truthfull."""
    # even thought it is a distance, we make it center around 0.5 instead of 0, to be compatible with the other metrics
    return ds['correct_truth_telling_base'] - ds['correct_truth_telling_adapt'] + 0.5


def undo_distance_truth_telling(base_more_truthful: Float[Tensor, ''], ds: Dataset, ):
    #note if we know which model is more truthful, and it's a binary choice, we can take the choice that is in that direction as the truth
    base_more_truthful = base_more_truthful - 0.5
    base_more_positive = (ds['binary_ans_base'] > ds['binary_ans_adapt']) * 1.0
    y = base_more_truthful * base_more_positive + (1-base_more_positive) * (-base_more_truthful)
    return y > 0.
    # return switch(base_more_positive, base_more_truthful)


y = distance_truth_telling(ds_val)
y1 = undo_distance_truth_telling(y, ds_val)
np.testing.assert_array_equal(y1, ds_val['label_true_base'], err_msg="undo_distance_truth_telling failed")


In [ ]:
# ds_val['label_true_base']


In [ ]:
ds2proxy = distance_truth_telling
proxy2label = undo_distance_truth_telling

# FIXME: the distance is 0.5 thresh but 0. Hmm


In [ ]:
from src.probes.pl_ranking_probe import InceptionBlock, LinBnDrop

    

class HSModel2d(nn.Module):
    def __init__(self, c_in, hs, dropout=0, depth=2):
        super().__init__()

        layers = [nn.BatchNorm1d(c_in[1], affine=False)]
        for i in range(depth+1):
            if (i>0) and (i<depth):
                layers.append(InceptionBlock(hs*4, hs, conv_dropout=dropout))
            elif i==0: # first layer
                if depth==0: 
                    layers.append(InceptionBlock(c_in[1], 1))
                else:
                    layers.append(InceptionBlock(c_in[1], hs, conv_dropout=dropout))
            else: # last layer
                layers.append(nn.Conv1d(hs*4, 1, 1))
        self.conv = nn.Sequential(*layers)
        
        n = c_in[0]
        self.head = nn.Sequential(
            LinBnDrop(n, n, p=dropout),
            LinBnDrop(n, n, p=dropout),
            nn.Linear(n, 1),  
        )

    def forward(self, x):
        x = x.float()
        x = rearrange(x, 'b l h -> b h l')
        x = self.conv(x)
        x = rearrange(x, 'b l h -> b (l h)')
        return self.head(x).squeeze(1)


In [ ]:
from src.helpers.sklearn_lightning import PLSKWrapper, PLSKBase
from torchmetrics.functional import accuracy

class PLSKMSErank(PLSKBase):
    def __init__(self, c_in, dropout=0, hs=32, depth=2, **kwargs):
        super().__init__(**kwargs)

        self.model = HSModel2d(c_in, hs, dropout)

    def forward(self, x):
        return self.model(x)

    def _step(self, batch, batch_idx, stage='train'):
        x, y = batch
        x0 = x[..., 0]
        x1 = x[..., 1]
        ypred0 = self(x0)
        ypred1 = self(x1)

        y_probs = F.sigmoid(ypred0-ypred1+0.5)
        y_cls = y_probs > 0.5
        
        if stage=='pred':
            return y_probs
        
        loss = F.smooth_l1_loss(ypred0-ypred1, y)
        
        y_cls = ypred1>ypred0 # switch2bool(ypred1-ypred0)
        self.log(f"{stage}/acc", accuracy(y_cls, y>0.5, "binary"), on_epoch=True, on_step=False)
        self.log(f"{stage}/loss", loss, on_epoch=True, on_step=True, prog_bar=True)
        self.log(f"{stage}/n", len(y), on_epoch=True, on_step=False, reduce_fx=torch.sum)
        return loss


In [ ]:
ds_train, ds_val = train_test_split_ds(ds_trainval, test_size=0.3)
X_val, y_val_proxy = ds2xy(ds_val, ds2proxy)
c_in = X_val.shape[1:]
c_in


In [ ]:
epoch_steps = int(len(ds_trainval)*0.7//batch_size)
pl_model = PLSKMSErank(c_in=c_in, epoch_steps=epoch_steps, max_epochs=max_epochs, depth=3, hs=64, dropout=0.1)


model = PLSKWrapper(pl_model, verbose=verbose, max_epochs=max_epochs)
if verbose: print(model)
df_res = eval_ranking(model, ds_trainval, ds_test, proxy2label, ds2proxy)
df_res2 = analyze_dfres(df_res)


### PL Direct classifier


In [ ]:

class HSModel3d(nn.Module):
    def __init__(self, c_in, hs, dropout=0, depth=2):
        super().__init__()
        self.preconv = nn.Sequential(
            nn.Conv2d(c_in[1], hs*4, (1, 2)),
        )

        layers = [nn.BatchNorm1d(hs*4, affine=False)]
        for i in range(depth+1):
            if (i>0) and (i<depth):
                layers.append(InceptionBlock(hs*4, hs, conv_dropout=dropout))
            elif i==0: # first layer
                if depth==0: 
                    layers.append(InceptionBlock(hs*4, 1))
                else:
                    layers.append(InceptionBlock(hs*4, hs, conv_dropout=dropout))
            else: # last layer
                layers.append(nn.Conv1d(hs*4, 1, 1))
        self.conv = nn.Sequential(*layers)
        
        n = c_in[0]
        self.head = nn.Sequential(
            LinBnDrop(n, n, p=dropout),
            LinBnDrop(n, n, p=dropout),
            nn.Linear(n, 1),  
        )

    def forward(self, x):
        x = x.float()
        if x.ndim==4:
            x = x.squeeze(3)
        x = rearrange(x, 'b l h v -> b h l v')
        x = self.preconv(x)
        x = rearrange(x, 'b h l v -> b h (l v)')
        x = self.conv(x)
        x = rearrange(x, 'b l h -> b (l h)')
        return self.head(x).squeeze(1)


In [ ]:
from src.helpers.sklearn_lightning import PLSKWrapper, PLSKBase
from torchmetrics.functional import accuracy

class PLSKDirect(PLSKBase):
    def __init__(self, c_in, dropout=0, hs=32, depth=2, **kwargs):
        super().__init__(**kwargs)

        self.model = HSModel3d(c_in, hs, dropout)

    def forward(self, x):
        return self.model(x)



In [ ]:
# set labels
ds2proxy = ranking_truth_telling
proxy2label = undo_ranked_truth_telling

# unit test labels
y_val1 = ds2proxy(ds_val)
y1 = proxy2label(y_val1, ds_val)
np.testing.assert_array_equal(y1, ds_val['label_true_base'], err_msg="undo_distance_truth_telling failed")
# assert (y1 == ds_val['label_true_base']).all()


In [ ]:
ds_train, ds_val = train_test_split_ds(ds_trainval, test_size=0.3)
X_val, y_val_proxy = ds2xy(ds_val, ds2proxy)
c_in = X_val.shape[1:]
c_in


In [ ]:
epoch_steps = int(len(ds_trainval)*0.7//batch_size)
pl_model = PLSKDirect(c_in=c_in, epoch_steps=epoch_steps, max_epochs=max_epochs, dropout=0.25, hs=16, depth=3)
if verbose: print(model)
model = PLSKWrapper(pl_model, verbose=verbose, max_epochs=max_epochs)
df_res = eval_ranking(model, ds_trainval, ds_test, proxy2label, ds2proxy)
df_res2 = analyze_dfres(df_res)


### Bool ranking


In [ ]:
from src.helpers.sklearn_lightning import PLSKWrapper, PLSKBase
from torchmetrics.functional import accuracy

class PLSKBoolRank(PLSKBase):
    def __init__(self, c_in, dropout=0, hs=32, depth=2, **kwargs):
        super().__init__(**kwargs)

        self.model = HSModel2d(c_in, hs, dropout)

    def forward(self, x):
        return self.model(x)

    def _step(self, batch, batch_idx, stage='train'):
        x, y = batch
        x0 = x[..., 0]
        x1 = x[..., 1]
        ypred0 = self(x0)
        ypred1 = self(x1)

        y_probs = F.sigmoid(ypred1-ypred0+0.5)
        y_cls = y_probs > 0.5
        
        if stage=='pred':
            return y_probs

        ranking_y = (y>0)*2-1 # from 0,1 to -1,1
        loss = F.margin_ranking_loss(ypred0, ypred1, ranking_y, margin=1)
        
        y_cls = ypred1>ypred0 # switch2bool(ypred1-ypred0)
        self.log(f"{stage}/acc", accuracy(y_cls, y>0.5, "binary"), on_epoch=True, on_step=False)
        self.log(f"{stage}/loss", loss, on_epoch=True, on_step=True, prog_bar=True)
        self.log(f"{stage}/n", len(y), on_epoch=True, on_step=False, reduce_fx=torch.sum)
        return loss


In [ ]:
ds_train, ds_val = train_test_split_ds(ds_trainval, test_size=0.3)
X_val, y_val_proxy = ds2xy(ds_val, ds2proxy)
c_in = X_val.shape[1:]
c_in


In [ ]:
epoch_steps = int(len(ds_trainval)*0.7//batch_size)
pl_model = PLSKBoolRank(c_in=c_in, epoch_steps=epoch_steps, max_epochs=max_epochs, depth=3, hs=64)


model = PLSKWrapper(pl_model, verbose=verbose, max_epochs=max_epochs)
if verbose: print(model)
df_res = eval_ranking(model, ds_trainval, ds_test, proxy2label, ds2proxy)
df_res2 = analyze_dfres(df_res)


### CCS

In [ ]:
from src.helpers.sklearn_lightning import PLSKWrapper, PLSKBase
from torchmetrics.functional import accuracy

def ccs_loss(p_neg, p_pos):
    consistency_losses = (p_pos - (1 - p_neg)) ** 2
    confidence_losses = torch.min(torch.stack((p_pos, p_neg), dim=-1), dim=-1).values ** 2
    return torch.mean(consistency_losses + confidence_losses)

class PLSKCCS(PLSKBase):
    def __init__(self, c_in, dropout=0, hs=32, depth=2, **kwargs):
        super().__init__(**kwargs)
        self.model = nn.Sequential(
            nn.Linear(np.prod(c_in), 1, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x).squeeze(1)

    def _step(self, batch, batch_idx, stage='train'):
        x, y = batch
        x = rearrange(x, 'b l h v -> b (l h v)').float()
        yb = y>0.5
        ypred = self(x)
        if stage=='pred':
            return ypred

        p_neg = ypred[~yb].mean()
        p_pos = ypred[yb].mean()
        loss = ccs_loss(p_neg, p_pos)
        assert torch.isfinite(loss)

        y_cls = ypred>0.5 # switch2bool(ypred1-ypred0)
        self.log(f"{stage}/acc", accuracy(y_cls, y>0.5, "binary"), on_epoch=True, on_step=False)
        self.log(f"{stage}/loss", loss, on_epoch=True, on_step=True, prog_bar=True)
        self.log(f"{stage}/n", len(y), on_epoch=True, on_step=False, reduce_fx=torch.sum)
        return loss


In [ ]:
batch_size = 256
epoch_steps = int(len(ds_trainval)*0.7//batch_size)
pl_model = PLSKCCS(c_in=c_in, epoch_steps=epoch_steps, max_epochs=max_epochs, depth=3, hs=64, lr=1e-4)


model = PLSKWrapper(pl_model, verbose=verbose, max_epochs=max_epochs, batch_size=batch_size)
if verbose: print(model)
df_res = eval_ranking(model, ds_trainval, ds_test, proxy2label, ds2proxy)
df_res2 = analyze_dfres(df_res)


In [ ]:
# CSS doesn't know the direction, so sometimes we need to flip it
df_res2['roc_auc'] = 1-df_res2['roc_auc']
df_res2['improvement'] = df_res2['roc_auc'] - df_res2['roc_auc_adapter']
df_res2.sort_values('improvement', ascending=False)


### Conformal

In [ ]:

import numpy as np
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB, MultinomialNB
from mapie.classification import MapieClassifier



In [ ]:
class MapieClassifier2(MapieClassifier):

    def fit(self, X_train, y_train, sample_weight=None, X_val=None, y_val=None, **kwargs):
        X_train = rearrange(X_train, 'b l h v -> b (l h v)')
        return super().fit(X_train.numpy(), y_train.numpy(), sample_weight, **kwargs)

    def predict_proba(self, X):
        X = rearrange(X, 'b l h v -> b (l h v)').numpy()
        return torch.from_numpy(super().predict(X)).float()


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced',)
model = MapieClassifier2(estimator=clf, 
                        # cv="prefit",
                        method="score",                        
                        )

df_res = eval_ranking(model, ds_trainval, ds_test, proxy2label, ds2proxy)
df_res2 = analyze_dfres(df_res)
